### Import libraries

In [ ]:
import pandas as pd
import time

### Loading Data

In [10]:
# Input path and filename
path = '../5. Master_thesis/Datasets/'
filename = path + 'Retails_ALL_CY06_CY22.xlsx'
external_file = path + 'External_indicators_extended.xlsx'

start_time = time.time()

# Load file into pandas dataframe
data_int = pd.concat(pd.read_excel(filename, sheet_name=None), ignore_index=True)
data_ext = pd.concat(pd.read_excel(external_file, sheet_name=None), ignore_index=True)

end_time = time.time()
time_elapsed = end_time - start_time 

print('Completed in {:0.2f} minutes'.format(time_elapsed/60))

Completed in 1.21 minutes


### Pre processing data

In [11]:
df = data_int.copy()
df_ext = data_ext.copy()

In [12]:
df

,orderNumber,RetailCountry,RetailDate,RetailCount
0,NNE0002970,FIN,01/02/2006 07:46:25,1
1,NNE0003175,NOR,01/02/2006 08:03:58,1
2,NNE0001616,DEN,01/02/2006 08:16:47,1
3,NNE0005269,FIN,01/02/2006 08:17:27,1
4,NNE0000178,FIN,01/02/2006 08:19:10,1
...,...,...,...,...
616295,NNE0633232,SWE,16/12/2022 06:13:03,1
616296,NNE0634926,NOR,26/12/2022 15:05:21,1
616297,NNE0632957,EST,30/12/2022 06:07:37,1
616298,NNE0632912,NOR,22/12/2022 13:38:44,1


In [13]:
# Remove potential extra spaces in cells using strip() function
df['RetailCountry'] = df['RetailCountry'].str.strip()
df['orderNumber'] = df['orderNumber'].str.strip()

# Sort by orderNumber and RetailDate
df.sort_values(by=['orderNumber', 'RetailDate'], inplace=True)

# Remove duplicates by orderNumber
df.drop_duplicates(subset=['orderNumber'], keep='last', inplace=True)

# Get only Finland orders
df = df[(df['RetailCountry'] == 'FIN')]

In [14]:
# Convert RetailDate to date
df['Date'] = pd.to_datetime(df['RetailDate'], dayfirst=True)
df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()

C:\Users\ne74255\AppData\Local\Temp/ipykernel_22228/1029513680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['RetailDate'], dayfirst=True)
C:\Users\ne74255\AppData\Local\Temp/ipykernel_22228/1029513680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()


In [15]:
# Generating index by date, grouping by and counting
df = df.groupby(['Date']).size().reset_index(name='Orders')

In [16]:
# Set index in both dataframes
df = df.set_index("Date")
df_ext = df_ext.set_index("Date")

# Merge both dataframes
data = pd.concat([df,df_ext], axis=1)

# Preview dataframe
data

,Orders,CCI,CPI,Unemployment_rate,Long_term_interest_rates,Avg_Fuel_Price,TIV
Date,,,,,,,
2006-01-01,1124,100.83240,0.807265,8.3,3.280000,1.170000,20177
2006-02-01,1079,100.80160,0.901804,8.0,3.440000,1.186667,12845
2006-03-01,1210,100.60100,0.899101,7.7,3.620000,1.180000,15163
2006-04-01,1147,100.48250,1.297405,7.7,3.880000,1.243333,13472
2006-05-01,1001,100.56730,1.701702,7.9,3.940000,1.260000,16516
...,...,...,...,...,...,...,...
2022-08-01,254,94.58021,7.616082,7.2,1.624904,2.153333,8163
2022-09-01,228,94.10754,8.119296,7.3,2.420836,2.133333,7867
2022-10-01,188,93.96142,8.310766,6.4,2.894486,2.136667,7132


In [17]:
# Save data to Excel so we don't need to run these steps every time
data.to_excel("Datasets/Output_files/0.xlsx") 

In [18]:
data

,Orders,CCI,CPI,Unemployment_rate,Long_term_interest_rates,Avg_Fuel_Price,TIV
Date,,,,,,,
2006-01-01,1124,100.83240,0.807265,8.3,3.280000,1.170000,20177
2006-02-01,1079,100.80160,0.901804,8.0,3.440000,1.186667,12845
2006-03-01,1210,100.60100,0.899101,7.7,3.620000,1.180000,15163
2006-04-01,1147,100.48250,1.297405,7.7,3.880000,1.243333,13472
2006-05-01,1001,100.56730,1.701702,7.9,3.940000,1.260000,16516
...,...,...,...,...,...,...,...
2022-08-01,254,94.58021,7.616082,7.2,1.624904,2.153333,8163
2022-09-01,228,94.10754,8.119296,7.3,2.420836,2.133333,7867
2022-10-01,188,93.96142,8.310766,6.4,2.894486,2.136667,7132
